<a href="https://colab.research.google.com/github/Profe-Jose-Burgos/Multi-Bot/blob/main/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Entrenamiento



In [ ]:
import json

In [ ]:
data = open("/content/drive/MyDrive/hackathon/tryouts.json").read()
intents = json.loads(data)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('wordnet')
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
words = []
classes = []
documents = []
ignore_words = ['?','!']

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
      x = nltk.word_tokenize(pattern)
      words.extend(x)
      documents.append((x, intent['tag']))

      if intent['tag'] not in classes:
        classes.append(intent['tag'])

words = [lemmatizer.lemmatize(x.lower()) for x in words if x not in ignore_words]

In [ ]:
import pickle

In [ ]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))


training = []
output_empty = [0] * len(classes)

for doc in documents:   
    count = []    
    pattern_words = doc[0]    
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    for w in words:
        count.append(1) if w in pattern_words else count.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([count, output_row])

In [ ]:
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

#Chat bot

In [ ]:
import random
import numpy as np
from tensorflow.keras.models import load_model

In [ ]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('/content/drive/MyDrive/Chat_bot/chat_bot_1.0/intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))